In [120]:
import scipy.stats as sts
import numpy as np
import pandas as pd

# Estimators
def nonParametricOptimalQuantity(distributionData, tau):
    optimalQuantity = np.quantile(distributionData,tau)
    return optimalQuantity

def profit(tau, demand, quantity):
    price = 1
    cost = 1 - tau
    return price * np.minimum(demand,quantity) - cost * quantity

def parametricOptimalQuantity(estimators, tau):
    optimalQuantity = sts.norm.ppf(tau, loc = estimators[0], scale = estimators[1])
    return optimalQuantity

# Evaluation of test statistics functions
def empericalRootMeanSquaredError(m, optimalQuantities, tau, meanDemand, stdevDemand):
    realOptimalQuantity = parametricOptimalQuantity([meanDemand, stdevDemand], tau)
    rmse = np.sqrt(1/m * np.sum((optimalQuantities - realOptimalQuantity)**2))
    return rmse

def empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand):
    return empericalRootMeanSquaredError(m, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand) / empericalRootMeanSquaredError(m, parametricOptimalQuantitys, tau, meanDemand, stdevDemand)

def empericalServiceLevel(m, optimalQuantities, demand):
    indicatorFunction = np.where(optimalQuantities.T >= np.array(demand), 1, 0)
    return 1 / m * np.sum(indicatorFunction)

def empericalProfitLoss(m, optimalQuantities, demand, tau, meanDemand, stdevDemand):
    realOptimalQuantity = parametricOptimalQuantity([meanDemand, stdevDemand], tau)
    profitReal = profit(tau, meanDemand, realOptimalQuantity)
    profitEstimator = profit(tau, demand, optimalQuantities.T)
    return 1 / m * np.sum(np.abs((profitReal - profitEstimator) /  profitReal))

def empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand):
    return empericalProfitLoss(m, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand) / empericalProfitLoss(m, parametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)

def monteCarlo(m, tau, n, meanDemand, stdevDemand):
    # Define arrays
    parametricOptimalQuantitys = np.empty((m,1))
    nonParametricOptimalQuantitys = np.empty((m,1))
    allParameters = np.empty((m,2))

    # Compute estimators and optimal quantities
    for j in range(m):
        distributionData = sts.norm.rvs(loc = meanDemand, scale = stdevDemand, size = n)
        allParameters[j] = parameters = sts.norm.fit(distributionData)
        parametricOptimalQuantitys[j] = parametricOptimalQuantity(parameters, tau)
        nonParametricOptimalQuantitys[j] = nonParametricOptimalQuantity(distributionData, tau)
    
    demand = allParameters[:,0] # in case of normal

    # Compute evaluation statistics
    rmse = empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand)
    eslParametric = empericalServiceLevel(m, parametricOptimalQuantitys, demand)
    eslNonParametric = empericalServiceLevel(m, nonParametricOptimalQuantitys, demand)
    eplr = empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)
    result = {
                    'MonteCarlo iterations' : m,
                    'Sample Size': n,
                    'Target Service Level': tau,
                    'Param Values': np.mean(allParameters, axis=0),
                    'RMSE Ratio': rmse,
                    'SL nonParam': eslParametric,
                    'SL Param': eslNonParametric,
                    'EPLR': eplr
                }
    return result





In [121]:
numberOfMontecarloIterations = 1000
stdevDemand = 15
meanDemand = 120

tauArray = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
nArray = [10, 50, 100, 200]

results = []
for tau in tauArray:
    for n in nArray:
        result = monteCarlo(numberOfMontecarloIterations, tau, n, meanDemand, stdevDemand)
        results.append(result)


In [122]:
df = pd.DataFrame(results)
df

,MonteCarlo iterations,Sample Size,Target Service Level,Param Values,RMSE Ratio,SL nonParam,SL Param,EPLR
0,1000,10,0.01,"[120.02958428317709, 13.83609571031229]",1.594529,0.0,0.000,1.730588
1,1000,50,0.01,"[120.09712800383289, 14.738975785604572]",1.756808,0.0,0.000,1.879300
2,1000,100,0.01,"[119.99377883107442, 14.928338285244449]",1.744289,0.0,0.000,1.792806
3,1000,200,0.01,"[119.99242205798646, 14.946755217338474]",1.892538,0.0,0.000,1.937648
4,1000,10,0.05,"[119.83504120928441, 13.813499792260636]",1.206407,0.0,0.000,1.220739
5,1000,50,0.05,"[119.89201103965544, 14.76635556010651]",1.289870,0.0,0.000,1.287659
6,1000,100,0.05,"[120.0243853639389, 14.906317988614434]",1.348998,0.0,0.000,1.338651
7,1000,200,0.05,"[120.04309062308829, 14.948952802082204]",1.315040,0.0,0.000,1.319465
8,1000,10,0.10,"[120.01739306648757, 13.887614106857244]",1.184578,0.0,0.000,1.203535
9,1000,50,0.10,"[119.96503456879417, 14.725124754014143]",1.211333,0.0,0.000,1.216465


In [123]:
import plotly.express as px

df_f = df[df["Sample Size"]==200]

fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, higher RMSE means parametric performs better")
fig.show()
